In [108]:
import os
import numpy as np
import pandas as pd

In [109]:
gov_train = pd.read_csv('Data/gove_train.csv', index_col = 0)

In [117]:
features = gov_train.columns[6:]
X = gov_train[features].select_dtypes(['number']).drop(columns = ['polrights_year', 'geff_year', 'rqual_year', 'rlaw_year'])
y = gov_train['gove']

In [118]:
import evalml
import woodwork as ww

problem_type = 'regression'
objective =  'root mean squared error'

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


automl = evalml.automl.AutoMLSearch(X_train, y_train, problem_type=problem_type, objective = objective)
automl.search()

best_pipeline = automl.best_pipeline

Using default limit of max_batches=1.

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for Root Mean Squared Error. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: catboost, xgboost, decision_tree, linear_model, extra_trees, random_forest, lightgbm



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mean Baseline Regression Pipeline        Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Root Mean Squared Error: 15.181
Batch 1: (2/9) Decision Tree Regressor w/ Imputer       Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Root Mean Squared Error: 3.004
Batch 1: (3/9) Extra Trees Regressor w/ Imputer         Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean Root Mean Squared Error: 2.138
Batch 1: (4/9) XGBoost Regressor w/ Imputer             Elapsed:00:02
	Starting cross validation
	Finished cross validation - mean Root Mean Squared Error: 1.581
Batch 1: (5/9) CatBoost Regressor w/ Imputer            Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean Root Mean Squared Error: 11.920
Batch 1: (6/9) Random Forest Regressor w/ Imputer       Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean Root Mean Squared Error: 2.232
Batch 1: (7/9) LightGBM Re

In [119]:
automl.rankings

,id,pipeline_name,score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,7,Linear Regressor w/ Imputer + Standard Scaler,1.492536,1.490913,90.168470,False,{'Imputer': {'categorical_impute_strategy': 'm...
1,3,XGBoost Regressor w/ Imputer,1.580945,1.543456,89.586107,False,{'Imputer': {'categorical_impute_strategy': 'm...
2,8,Elastic Net Regressor w/ Imputer + Standard Sc...,1.901795,1.824356,87.472632,False,{'Imputer': {'categorical_impute_strategy': 'm...
3,2,Extra Trees Regressor w/ Imputer,2.138320,2.130133,85.914606,False,{'Imputer': {'categorical_impute_strategy': 'm...
4,5,Random Forest Regressor w/ Imputer,2.231907,2.051489,85.298141,False,{'Imputer': {'categorical_impute_strategy': 'm...
5,6,LightGBM Regressor w/ Imputer,2.942094,2.825164,80.620041,False,{'Imputer': {'categorical_impute_strategy': 'm...
6,1,Decision Tree Regressor w/ Imputer,3.003826,2.974130,80.213408,False,{'Imputer': {'categorical_impute_strategy': 'm...
7,4,CatBoost Regressor w/ Imputer,11.919868,12.162181,21.482281,False,{'Imputer': {'categorical_impute_strategy': 'm...
8,0,Mean Baseline Regression Pipeline,15.181118,15.494977,0.000000,False,{'Baseline Regressor': {'strategy': 'mean'}}


In [120]:
best_pipeline.graph()

RuntimeError: To graph entity sets, a graphviz backend is required.
Install the backend using one of the following commands:
  Mac OS: brew install graphviz
  Linux (Ubuntu): sudo apt-get install graphviz
  Windows: conda install python-graphviz


In [121]:
best_pipeline.feature_importance

,feature,importance
0,polrights,2.901770
1,judi,2.884822
2,rlaw,2.879397
3,geff,2.465687
4,polity2,2.317463
5,ticpi,2.200867
6,elfcr,1.763376
7,vapvt,1.327586
8,womenparl,0.860494
9,trang,0.361687


In [123]:
best_pipeline.graph_feature_importance()

In [124]:
predictions = best_pipeline.predict(X_test)
predictions = predictions.to_series()

In [125]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('R2: ' + str(r2_score(y_test,  predictions)))
print('MSE: ' + str(mean_squared_error(y_test, predictions, squared = True)))
print('RMSE: ' + str(mean_squared_error(y_test, predictions, squared = False)))
print('MAE: ' + str(mean_absolute_error(y_test, predictions)))

R2: 0.9916054331656143
MSE: 1.986364637621028
RMSE: 1.409384488924519
MAE: 1.134158766020175


In [126]:
best_pipeline.hyperparameters

{'Imputer': {'categorical_impute_strategy': ['most_frequent'],
  'numeric_impute_strategy': ['mean', 'median', 'most_frequent']},
 'Standard Scaler': {},
 'Linear Regressor': {'fit_intercept': [True, False],
  'normalize': [True, False]}}